# 1. Prediction


In [1]:
import tensorflow as tf
from eval_utils import *
%matplotlib inline

print(tf.__version__)

2.4.2


In [2]:
! nvidia-smi

/bin/bash: switchml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `switchml'
/bin/bash: _moduleraw: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `_moduleraw'
Thu Sep  9 12:05:03 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.142.00   Driver Version: 450.142.00   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   40C    P0    26W /  70W |      0MiB / 15109MiB |      0%    

## 1.1 Load Image

In [3]:
import os

SRC = "/home/ubuntu/crack/Origin-New"
image_list = sorted([file for file in os.listdir(SRC) if file.endswith("JPG")])

process_image(SRC, image_list)

In [4]:
images = load_image((224,224))
images.shape

(1500, 224, 224, 1)

## 1.2 Prediction - Binomial

In [5]:
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet152V2, DenseNet201
from tensorflow.keras.applications.densenet import preprocess_input as pre_densenet
from tensorflow.keras.applications.resnet_v2 import preprocess_input as pre_resnet_v2

model = DenseNet201()
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

features = extract_in_batch(10, images, model, pre_densenet)
features.shape

Extension horovod.torch has not been built: /home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/horovod/torch/mpi_lib/_mpi_lib.cpython-37m-x86_64-linux-gnu.so not found
If this is not expected, reinstall Horovod with HOROVOD_WITH_PYTORCH=1 to debug the build error.
Warning! MPI libs are missing, but python applications are still avaiable.
[2021-09-09 12:05:26.508 ip-172-31-5-51:14743 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-09-09 12:05:26.530 ip-172-31-5-51:14743 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.


(1500, 1920)

In [6]:
from tensorflow.keras.models import load_model

model = load_model('model/binary_TL_DenseNet201.h5')
y_pred_bi = np.where(model.predict(features) > 0.5, 1, 0).flatten()
y_pred_bi = np.array(decode(y_pred_bi))

## 1.3 Prediction - Multinomial

### 1.3.1 ResNet152V2

In [7]:
model = ResNet152V2()
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

features = extract_in_batch(10, images, model, pre_resnet_v2)
features.shape

(1500, 2048)

In [8]:
model = load_model('model/multi_TL_ResNet152V2.h5')
y_pred_mul_1 = (np.argmax(model.predict(features), axis=1) + 2).flatten()
y_pred_mul_1 = np.array(decode(y_pred_mul_1))

y_pred_mul_1[y_pred_bi == "none"] = "none"

### 1.3.2 DenseNet201

In [9]:
model = DenseNet201()
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

features = extract_in_batch(10, images, model, pre_densenet)
features.shape

(1500, 1920)

In [10]:
model = load_model('model/multi_TL_DenseNet201.h5')
y_pred_mul_2 = (np.argmax(model.predict(features), axis=1) + 2).flatten()
y_pred_mul_2 = np.array(decode(y_pred_mul_2))

y_pred_mul_2[y_pred_bi == "none"] = "none"

# 2. Evaluation

In [11]:
import pandas as pd 
import numpy as np

data = pd.read_csv("labels.csv", index_col=0)
data.head()

,crack_1,crack_2,crack_3
image_name,,,
06001.JPG,none,none,none
06002.JPG,none,long,long
06003.JPG,none,none,none
06004.JPG,none,none,none
06005.JPG,none,none,none


In [12]:
y_true = {'crack_type' : data.values.reshape(-1,)}
y_true = pd.DataFrame(y_true, index=np.repeat(data.index, len(data.columns)))["crack_type"]
y_true[y_true == "RAIl"] = "rail"

## 2.1 Binary Accuracy

In [13]:
y_true_bi = y_true.copy()
y_true_bi[y_true != "none"] = "crack"
get_result(y_true_bi, y_pred_bi, False)

              precision    recall  f1-score   support

       crack       0.65      0.95      0.77       297
        none       0.99      0.87      0.93      1203

    accuracy                           0.89      1500
   macro avg       0.82      0.91      0.85      1500
weighted avg       0.92      0.89      0.90      1500

Test accuracy: 0.89


## 2.2 Multi-nomial Accuracy

In [14]:
get_result(y_true[y_true != "none"], y_pred_mul_1[y_true != "none"])

              precision    recall  f1-score   support

        croc       0.85      0.68      0.76       145
        diag       1.00      0.12      0.22        16
         lat       0.00      0.00      0.00        17
        long       0.59      0.69      0.64       118
        none       0.00      0.00      0.00         0
        rail       0.25      1.00      0.40         1

    accuracy                           0.62       297
   macro avg       0.45      0.42      0.34       297
weighted avg       0.71      0.62      0.64       297

Test accuracy: 0.62
Test f1 micro: 0.62
Test f1 macro: 0.34
Test f1 weighted: 0.64


In [15]:
get_result(y_true[y_true != "none"], y_pred_mul_2[y_true != "none"])

              precision    recall  f1-score   support

        croc       0.94      0.76      0.84       145
        diag       0.50      0.44      0.47        16
         lat       0.09      0.29      0.14        17
        long       0.73      0.57      0.64       118
        none       0.00      0.00      0.00         0
        rail       0.25      1.00      0.40         1

    accuracy                           0.64       297
   macro avg       0.42      0.51      0.41       297
weighted avg       0.78      0.64      0.70       297

Test accuracy: 0.64
Test f1 micro: 0.64
Test f1 macro: 0.41
Test f1 weighted: 0.70


## 2.2 Overall Accuracy

In [16]:
get_result(y_true, y_pred_mul_1)

              precision    recall  f1-score   support

        croc       0.73      0.68      0.71       145
        diag       0.50      0.12      0.20        16
         lat       0.00      0.00      0.00        17
        long       0.37      0.69      0.48       118
        none       0.99      0.87      0.93      1203
        rail       0.20      1.00      0.33         1

    accuracy                           0.82      1500
   macro avg       0.46      0.56      0.44      1500
weighted avg       0.90      0.82      0.85      1500

Test accuracy: 0.82
Test f1 micro: 0.82
Test f1 macro: 0.44
Test f1 weighted: 0.85


In [17]:
get_result(y_true, y_pred_mul_2)

              precision    recall  f1-score   support

        croc       0.79      0.76      0.77       145
        diag       0.33      0.44      0.38        16
         lat       0.04      0.29      0.07        17
        long       0.46      0.57      0.51       118
        none       0.99      0.87      0.93      1203
        rail       0.12      1.00      0.22         1

    accuracy                           0.83      1500
   macro avg       0.46      0.66      0.48      1500
weighted avg       0.91      0.83      0.86      1500

Test accuracy: 0.83
Test f1 micro: 0.83
Test f1 macro: 0.48
Test f1 weighted: 0.86


## 2.3 Prediction Distribution

In [18]:
pd.Series(y_pred_mul_1).value_counts() 

none    1066
long     223
croc     135
lat       67
rail       5
diag       4
dtype: int64

In [19]:
pd.Series(y_pred_mul_2).value_counts() 

none    1066
long     146
croc     139
lat      120
diag      21
rail       8
dtype: int64

In [20]:
pd.Series(data.values.reshape(-1,)).value_counts() 

none    1203
croc     145
long     118
lat       17
diag      16
RAIl       1
dtype: int64